In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 12 01:09:17 2016

@author: stephen
"""

from tensorflow import keras
import numpy as np
import pandas as pd

np.random.seed(813306)


def build_resnet(input_shape, n_feature_maps, nb_classes):
    print("build conv_x")
    x = keras.layers.Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv2D(n_feature_maps, 8, 1, padding="same")(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation("relu")(conv_x)

    print("build conv_y")
    conv_y = keras.layers.Conv2D(n_feature_maps, 5, 1, padding="same")(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation("relu")(conv_y)

    print("build conv_z")
    conv_z = keras.layers.Conv2D(n_feature_maps, 3, 1, padding="same")(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps, 1, 1, padding="same")(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)
    print("Merging skip connection")
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation("relu")(y)

    print("build conv_x")
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps * 2, 8, 1, padding="same")(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation("relu")(conv_x)

    print("build conv_y")
    conv_y = keras.layers.Conv2D(n_feature_maps * 2, 5, 1, padding="same")(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation("relu")(conv_y)

    print("build conv_z")
    conv_z = keras.layers.Conv2D(n_feature_maps * 2, 3, 1, padding="same")(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps * 2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps * 2, 1, 1, padding="same")(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print("Merging skip connection")
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation("relu")(y)

    print("build conv_x")
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps * 2, 8, 1, padding="same")(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation("relu")(conv_x)

    print("build conv_y")
    conv_y = keras.layers.Conv2D(n_feature_maps * 2, 5, 1, padding="same")(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation("relu")(conv_y)

    print("build conv_z")
    conv_z = keras.layers.Conv2D(n_feature_maps * 2, 3, 1, padding="same")(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps * 2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps * 2, 1, 1, padding="same")(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print("Merging skip connection")
    y = keras.layers.Add()([shortcut_y, conv_z])
    y = keras.layers.Activation("relu")(y)

    full = keras.layers.GlobalAveragePooling2D()(y)
    out = keras.layers.Dense(nb_classes, activation="softmax")(full)
    print("        -- model was built.")
    return x, out


def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:, 0]
    X = data[:, 1:]
    return X, Y


# epochs = 1500
epochs = 10

flist = ["../../datasets/MixedShapesSmallTrain/MixedShapesSmallTrain"]
for each in flist:
    fname = each
    x_train, y_train = readucr(fname + "_TRAIN.txt")
    x_test, y_test = readucr(fname + "_TEST.txt")
    nb_classes = len(np.unique(y_test))
    batch_size = int(min(x_train.shape[0] / 10, 16))

    y_train = (
        (y_train - y_train.min()) / (y_train.max() - y_train.min()) * (nb_classes - 1)
    )
    y_test = (y_test - y_test.min()) / (y_test.max() - y_test.min()) * (nb_classes - 1)

    Y_train = keras.utils.to_categorical(y_train, nb_classes)
    Y_test = keras.utils.to_categorical(y_test, nb_classes)

    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean) / (x_train_std)

    x_test = (x_test - x_train_mean) / (x_train_std)
    x_train = x_train.reshape(
        x_train.shape
        + (
            1,
            1,
        )
    )
    x_test = x_test.reshape(
        x_test.shape
        + (
            1,
            1,
        )
    )

    x, y = build_resnet(x_train.shape[1:], 64, nb_classes)
    model = keras.models.Model(inputs=x, outputs=y)
    optimizer = keras.optimizers.Adam()
    model.compile(
        loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"]
    )

    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="loss", factor=0.5, patience=50, min_lr=0.0001
    )
    hist = model.fit(
        x_train,
        Y_train,
        batch_size=batch_size,
        epochs=epochs,
        verbose=1,
        validation_data=(x_test, Y_test),
        callbacks=[reduce_lr],
    )

build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
        -- model was built.
Epoch 1/10
10/10 [==============================] - 139s 15s/step - loss: 1.5077 - accuracy: 0.3000 - val_loss: 1.5942 - val_accuracy: 0.2697 - lr: 0.0010
Epoch 2/10
10/10 [==============================] - 185s 20s/step - loss: 1.2735 - accuracy: 0.4800 - val_loss: 1.5579 - val_accuracy: 0.3159 - lr: 0.0010
Epoch 3/10
10/10 [==============================] - 137s 15s/step - loss: 1.1730 - accuracy: 0.5600 - val_loss: 1.5380 - val_accuracy: 0.3340 - lr: 0.0010
Epoch 4/10
10/10 [==============================] - 140s 15s/step - loss: 1.1396 - accuracy: 0.5000 - val_loss: 1.5416 - val_accuracy: 0.2697 - lr: 0.0010
Epoch 5/10
10/10 [==============================] - 127s 14s/step - loss: 1.1557 - accuracy: 0.5500 - val_loss: 1.5898 - val_accuracy: 0.2697 - lr: 0.0010
Epoch 6/